In [ ]:
from openai import OpenAI
import os 

api_key = os.environ['OPENAI_API_KEY']
client = OpenAI(api_key=api_key)



In [ ]:
import pandas as pd

df = pd.read_csv("./data/job_description.csv")

In [ ]:
job_des = df['Job_Description'].tolist()

In [ ]:
job_des[0]

In [ ]:

from supabase import create_client
def setup():
    client = create_client(
        os.environ.get("SUPABASE_URL"),
        os.environ.get("SUPABASE_KEY")
    )
    return client

supabase = setup()


In [ ]:
for i in range(1, len(job_des)):
    print(i)
    st = job_des[i]
    response = client.embeddings.create(
        input=st,
        model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    response = {}
    response['embedding'] = embedding
    response['description'] = st
    data, count = supabase.table("embeddings").insert(response).execute()